In [28]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [30]:
ol = Overlay("s_r_int.bit")

In [33]:
ol?

In [34]:
inBuffer0 = allocate(shape=(784,), dtype=np.int32)
outBuffer= allocate(shape=(10,), dtype=np.int32)
dma=ol.axi_dma_0


In [125]:
inBuffer0[:]=np.ones(784)*1

In [126]:
dma.sendchannel.transfer(inBuffer0)
dma.recvchannel.transfer(outBuffer)
ol.top_0.write(0x00, 0x01)
while(ol.top_0.register_map.CTRL.AP_DONE==0):
    pass
print(outBuffer)


0
[-4126 -5920 -2955 -7247  1155  3766 -6784 -5808  -952  1640]
0


In [127]:
label=np.load("label.npy")

In [38]:
img=np.load("img.npy")

In [43]:
#turn float to int
for i in range(1000):
    for j in range(784):
        img[i,j]=int((img[i,j]+1)/2*255-127)
    

In [131]:
import time

In [132]:
hit=0
start_time=time.time()
for i in range(1000):
    inBuffer0[:]=img[i]
    dma.sendchannel.transfer(inBuffer0)
    dma.recvchannel.transfer(outBuffer)
    ol.top_0.write(0x00, 0x01)
    while(ol.top_0.register_map.CTRL.AP_DONE==0):
        pass
    if(outBuffer.argmax()==label[i]):
        hit+=1
    #print("times: ",i+1,"hit: ",hit)
print("acc",hit/1000)
print("spend time",time.time()-start_time)

acc 0.966
spend time 14.211108684539795
